In [1]:
%load_ext dotenv

%dotenv ../tests/.env.test
%dotenv -o ../.env


In [2]:
from uuid import uuid4
import json


import redbox
from redbox.app import Redbox
from redbox.models.chain import RedboxQuery, RedboxState


In [3]:

app = Redbox(debug=True)

INFO:elastic_transport.transport:HEAD http://localhost:9200/_alias/redbox-data-chunk-current [status:200 duration:0.004s]
/Users/james.richards/projects/redbox-copilot/redbox-core/venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/james.richards/projects/redbox-copilot/redbox-core/venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/jam

In [4]:
s = RedboxState(
    request=RedboxQuery(
        question="@gadget on what day was the first moon landing?",
        s3_keys=[],
        user_uuid="5210311c-df38-430f-8451-df5fa28f0be3", # Get this from the URL in the admin console while viewing your local superuser
        chat_history=[],
        permitted_s3_keys=[]
    )
) 


In [5]:

response_text = ""
citations = []
activity_log = []
async def token_handler(s):
    global response_text
    response_text += s
async def citations_handler(cs):
    citations.extend(cs)
async def activity_handler(e):
    activity_log.append(e)

r = await app.run(
    input=s,
    response_tokens_callback=token_handler,
    citations_callback=citations_handler,
    activity_event_callback=activity_handler
)

print("*"*10)
print()
print(f"User: {r['request'].question}")
print()
print("Activity Log:")
for l in activity_log:
    print(f"\t - {l.message}")
print()
print(f"AI: {response_text}")
print()
print("Citations: ")
for c in citations:
    print(c.text_in_answer)
    for source in c.sources:
        print(f"[{source.source_type}] {source.source}")
        print("\t" + source.highlighted_text_in_source)


INFO:elastic_transport.transport:POST http://localhost:9200/redbox-data-chunk-current/_search?scroll=5m [status:200 duration:0.002s]
INFO:elastic_transport.transport:DELETE http://localhost:9200/_search/scroll [status:200 duration:0.001s]
/Users/james.richards/projects/redbox-copilot/redbox-core/redbox/chains/components.py:31: LangChainBetaWarning: The function `init_chat_model` is in beta. It is actively being worked on, so the API may change.
  chat_model = init_chat_model(
INFO:httpx:HTTP Request: POST https://oai-i-dot-ai-playground-sweden.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:redbox.graph.nodes.processes:Invoking tool _search_wikipedia with args {'query': 'first moon landing date'}
INFO:httpx:HTTP Request: POST https://oai-i-dot-ai-playground-sweden.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://oai-i-dot-ai-playground-sw

**********

User: @gadget on what day was the first moon landing?

Activity Log:
	 - You selected no files
	 - Searching Wikipedia for 'first moon landing date'
	 - Reading Wikipedia page Apollo_11

AI: The first moon landing occurred on July 20, 1969.

Citations: 
The first moon landing occurred on July 20, 1969.
[URL] https://en.wikipedia.org/wiki/Apollo_11
	Commander Neil Armstrong and Lunar Module Pilot Buzz Aldrin landed the Apollo Lunar Module Eagle on July 20, 1969, at 20:17 UTC
